<a href="https://colab.research.google.com/github/muhammadnadhir6/Project-Portofolio/blob/main/User_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [ ]:
#Import library
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats
from operator import attrgetter
import plotly.express as px
import seaborn as sns

# Import Data from Google Drive to DataFrame

In [ ]:
#Read csv from public google drive
url = 'https://drive.google.com/file/d/1iVi5k9OJ2xobQTglVNQ5vExqBcF27cF0/view?usp=drive_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

# Data Preview

In [ ]:
#Table information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


In [ ]:
df.describe()

,quantity,price,customer_id
count,461773.000000,461773.000000,360853.000000
mean,9.088892,4.569963,15557.611138
std,86.366467,153.411566,1593.728741
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14210.000000
50%,3.000000,2.100000,15580.000000
75%,10.000000,4.210000,16938.000000
max,10200.000000,25111.090000,18287.000000


In [ ]:
#Table preview
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


# Data Cleansing

In [ ]:
#Copying table
df_clean = df.copy()

In [ ]:
#Drop rows with null value
df_clean = df_clean.dropna()

In [ ]:
#Converting order date column data type into datetime
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64')

In [ ]:
#Transforming product name string into lower character
df_clean['product_name'] = df_clean['product_name'].str.lower()

In [ ]:
#Drop test rows based on product code and product name
df_clean = df_clean[df_clean['product_name'].str.contains('|'.join(['test', 'adjust']))==False]

In [ ]:
#Changing quantity with negative value into positive because it indicates cancelled order
df_clean['quantity'] = df_clean['quantity'].abs()

<ipython-input-11-d70cfb2cbcb5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['quantity'] = df_clean['quantity'].abs()


In [ ]:
#Remove rows with zero or negative price value
df_clean = df_clean[df_clean['price']>0]

In [ ]:
#Make amount column by multiply price and quantity
df_clean['amount'] = df_clean['price']*df_clean['quantity']

<ipython-input-13-eb0a10f38ecc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['amount'] = df_clean['price']*df_clean['quantity']


In [ ]:
#Make order status column to indicate cancelled or delivered order based on order id
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')

In [ ]:
#Rename product name for each product id with product name with the most order count
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')

In [ ]:
#Convert the data type of customer id into string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)

In [ ]:
#Remove outlier
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)

In [ ]:
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,amount,order_status
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01-04,4.25,cancelled
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,91.80,delivered
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,51.00,delivered
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,5.10,delivered
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,8.50,delivered
...,...,...,...,...,...,...,...,...,...,...
358435,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12-23,1.25,delivered
358436,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,1.69,delivered
358437,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,1.69,delivered
358438,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12-23,9.90,delivered


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358440 entries, 0 to 358439
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      358440 non-null  object        
 1   product_code  358440 non-null  object        
 2   product_name  358440 non-null  object        
 3   quantity      358440 non-null  int64         
 4   order_date    358440 non-null  object        
 5   price         358440 non-null  float64       
 6   customer_id   358440 non-null  object        
 7   date          358440 non-null  datetime64[ns]
 8   amount        358440 non-null  float64       
 9   order_status  358440 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 27.3+ MB


# RFM Segmentation

### Count total order, latest order date, and total order value for each customer

In [ ]:
df_user = df_clean.groupby('customer_id', as_index=False).agg(order_cnt=('order_id', 'nunique'), max_order_date=('date', 'max'), total_order_value=('amount', 'sum'))
df_user

,customer_id,order_cnt,max_order_date,total_order_value
0,12346.0,4,2010-10-04,498.90
1,12608.0,1,2010-10-31,415.79
2,12745.0,2,2010-08-10,723.85
3,12746.0,2,2010-06-30,266.35
4,12747.0,19,2010-12-13,4094.79
...,...,...,...,...
3876,18283.0,6,2010-11-22,641.77
3877,18284.0,2,2010-10-06,486.68
3878,18285.0,1,2010-02-17,427.00
3879,18286.0,2,2010-08-20,941.48


### Calculate the gap between the latest order of each customer with the latest order in the dataset

In [ ]:
today = df_clean['date'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12346.0,4,2010-10-04,498.90,80
1,12608.0,1,2010-10-31,415.79,53
2,12745.0,2,2010-08-10,723.85,135
3,12746.0,2,2010-06-30,266.35,176
4,12747.0,19,2010-12-13,4094.79,10
...,...,...,...,...,...
3876,18283.0,6,2010-11-22,641.77,31
3877,18284.0,2,2010-10-06,486.68,78
3878,18285.0,1,2010-02-17,427.00,309
3879,18286.0,2,2010-08-20,941.48,125


##Recency

### Calculate recency score for each customer based on gap since last order (the lower the gap, the higher the recency score)

In [ ]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score
0,12346.0,4,2010-10-04,498.90,80,2
1,12608.0,1,2010-10-31,415.79,53,3
2,12745.0,2,2010-08-10,723.85,135,2
3,12746.0,2,2010-06-30,266.35,176,1
4,12747.0,19,2010-12-13,4094.79,10,5
...,...,...,...,...,...,...
3876,18283.0,6,2010-11-22,641.77,31,4
3877,18284.0,2,2010-10-06,486.68,78,2
3878,18285.0,1,2010-02-17,427.00,309,1
3879,18286.0,2,2010-08-20,941.48,125,2


##Frequency

### Calculate frequency score for each customer based on their order count (the more the order count, the higher the frequency score)

In [ ]:
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=[0,
                                          np.percentile(df_user['order_cnt'], 20),
                                          np.percentile(df_user['order_cnt'], 40),
                                          np.percentile(df_user['order_cnt'], 60),
                                          np.percentile(df_user['order_cnt'], 80),
                                          df_user['order_cnt'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score
0,12346.0,4,2010-10-04,498.90,80,2,3
1,12608.0,1,2010-10-31,415.79,53,3,1
2,12745.0,2,2010-08-10,723.85,135,2,2
3,12746.0,2,2010-06-30,266.35,176,1,2
4,12747.0,19,2010-12-13,4094.79,10,5,5
...,...,...,...,...,...,...,...
3876,18283.0,6,2010-11-22,641.77,31,4,4
3877,18284.0,2,2010-10-06,486.68,78,2,2
3878,18285.0,1,2010-02-17,427.00,309,1,1
3879,18286.0,2,2010-08-20,941.48,125,2,2


##Monetary

### Calculate frequency score for each customer based on their total order value (the higher the value, the higher the monetary score)

In [ ]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                   bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score
0,12346.0,4,2010-10-04,498.90,80,2,3,3
1,12608.0,1,2010-10-31,415.79,53,3,1,2
2,12745.0,2,2010-08-10,723.85,135,2,2,3
3,12746.0,2,2010-06-30,266.35,176,1,2,2
4,12747.0,19,2010-12-13,4094.79,10,5,5,5
...,...,...,...,...,...,...,...,...
3876,18283.0,6,2010-11-22,641.77,31,4,4,3
3877,18284.0,2,2010-10-06,486.68,78,2,2,3
3878,18285.0,1,2010-02-17,427.00,309,1,1,2
3879,18286.0,2,2010-08-20,941.48,125,2,2,4


##Segmentation

###Define segment for each customer based on their recency and frequency score

In [ ]:
df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating']
)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score,segment
0,12346.0,4,2010-10-04,498.90,80,2,3,3,08-At Risk
1,12608.0,1,2010-10-31,415.79,53,3,1,2,09-About to Sleep
2,12745.0,2,2010-08-10,723.85,135,2,2,3,10-Hibernating
3,12746.0,2,2010-06-30,266.35,176,1,2,2,10-Hibernating
4,12747.0,19,2010-12-13,4094.79,10,5,5,5,01-Champion
...,...,...,...,...,...,...,...,...,...
3876,18283.0,6,2010-11-22,641.77,31,4,4,3,02-Loyal Customers
3877,18284.0,2,2010-10-06,486.68,78,2,2,3,10-Hibernating
3878,18285.0,1,2010-02-17,427.00,309,1,1,2,10-Hibernating
3879,18286.0,2,2010-08-20,941.48,125,2,2,4,10-Hibernating


###Segmentation Summary

In [ ]:
summary = pd.pivot_table(df_user, index='segment',
               values=['customer_id','day_since_last_order','order_cnt','total_order_value'],
               aggfunc={'customer_id': pd.Series.nunique,
                        'day_since_last_order': [np.mean, np.median],
                        'order_cnt': [np.mean, np.median],
                        'total_order_value': [np.mean, np.median]})
summary['pct_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)
summary

customer_id day_since_last_order         order_cnt  \
                           nunique                 mean median       mean   
segment                                                                     
01-Champion                    550            10.618182    9.5  15.454545   
02-Loyal Customers             543            40.872928   37.0   8.784530   
03-Potential Loyalists         524            23.566794   24.0   2.833969   
04-Can't Lose Them              63           122.698413  113.0  11.428571   
05-Need Attention              177            58.655367   59.0   3.401130   
06-New Customers                50            14.220000   16.0   1.000000   
07-Promising                   143            32.755245   34.0   1.000000   
08-At Risk                     426           140.044601  120.0   4.138498   
09-About to Sleep              352            58.735795   58.0   1.417614   
10-Hibernating                1053           195.870845  198.0   1.314340   

                              total_order_value           pct_unique  
                       median              mean    median             
segment                                                               
01-Champion              10.0       5003.131991  2775.525       14.2  
02-Loyal Customers        7.0       2627.873376  1956.650       14.0  
03-Potential Loyalists    3.0        768.504733   625.710       13.5  
04-Can't Lose Them       10.0       2859.818254  2269.460        1.6  
05-Need Attention         3.0        997.686559   828.550        4.6  
06-New Customers          1.0        244.689000   193.675        1.3  
07-Promising              1.0        287.031888   237.420        3.7  
08-At Risk                4.0       1156.206974   876.450       11.0  
09-About to Sleep         1.0        448.229688   334.755        9.1  
10-Hibernating            1.0        345.084704   258.600       27.1

In [ ]:
segment = df_user.groupby('segment', as_index=False).agg(seg_count=('segment', 'count'))

In [ ]:
segment

,segment,seg_count
0,01-Champion,550
1,02-Loyal Customers,543
2,03-Potential Loyalists,524
3,04-Can't Lose Them,63
4,05-Need Attention,177
5,06-New Customers,50
6,07-Promising,143
7,08-At Risk,426
8,09-About to Sleep,352
9,10-Hibernating,1053


In [ ]:
fig = px.histogram(segment, x='segment',
                y='seg_count',
                color='segment',
                title = 'Customer Segmentation',
                labels = dict(x = "Customer Category", y ="Counts"))
fig.show()

#Insight

- Most of the users are in Hibernating segment (1053 or 27.1%), Champion (550 or 14.2%), and Loyal Customers (543 or 14%)
- We can make a special program for Loyal Customers that can push them to make another transaction as soon as possible so we can make them into a Champion
- We can also make a special program for Potential Loyalist that can push them make a transaction more frequently so we can make them into a Champion
- A special program for Hibernating customers can be made to make them come back again so they can be a New Customers or even a Potential Loyalist